In [ ]:
# Tutorial for a standard mission simulation described in Airbus: Getting to grips in aircraft performance
# Aircraft: Boeing 737-800
# 
# Mission_Simulation_TUT.py
##
# Created:  Sep 2022, R. ROJAS CARDENAS
#

# ----------------------------------------------------------------------
#   Imports
# ----------------------------------------------------------------------

# General Python Imports
import logging
logging.basicConfig(filename='Simulation_loggings.log', encoding='utf-8', level=logging.DEBUG)
import numpy as np
import pandas as pd
import plotly.express as px
from re import search
import time
import ipywidgets as widgets
from IPython.display import clear_output, display
import plotly.graph_objs as go

##CosApp
from cosapp.base import System
#Important to define a path directory for other modules either in the enviroment or with 'sys' method.

## Modules and tools
from amad.disciplines.performance.systems import missionProfile #Module to define the mission segments.
import amad.disciplines.aerodynamics.tools.createAeroInterpolationCSV as aeroInterp #Tool to create the function to Interpolate.
import amad.tools.unit_conversion as uc
from amad.disciplines.design.resources.aircraft_geometry_library import ac_narrow_body_long_opti as airplane_geom
#Geometry of the aircraft to use in the simulation.

def print_callback(callback_data):
    print(callback_data)

live_figure = go.FigureWidget()
live_figure.add_scatter()
live_figure.update_layout(xaxis = dict(range=[0,1000000]), yaxis= dict(range=[0,10000]))
live_data_x = []
live_data_y = []

last_point_x = 0.
last_point_y = 0.

def live_graph_callback(callback_data):
    global last_point_x
    datapoint_x = callback_data['data']['position'][0]
    datapoint_y = callback_data['data']['position'][2]

    if abs(datapoint_x - last_point_x) > 2000:
        live_data_x.append(datapoint_x)
        live_data_y.append(datapoint_y)
        last_point_x = datapoint_x

    live_figure.data[0].x = live_data_x
    live_figure.data[0].y = live_data_y

class Mission_simulation(System):
    """ The simulation performs the following tasks:
        1) The simulation considers those segments defined in the mission_profile module and the values
        to respect under each segment.
        2) The module computes the point at which the AC shall start the descent using connections with the cruise segment variables.
        3) The module has a function to create an interactive plot to visualize the results along with the events.
        4) The module can export the module in a CSV file.
        
        Source: Inspired in FAST-GA data post processing methods.
        None
    """
    
    def __init__(self,*args, **kwargs):
        
        self.Mission  = None #Variable saving Mission object.
        
        self.data={} #Dict to save the dataframes containing the results from each simulation.
        
        self._fig = None # The figure displayed
        self._x_widget = None # The x selector
        self._y_widget = None # The y selector
        self._segment_widget = None # The segment selector
        self.add_parameter_widget = None # The  selector for an extra data info

        super().__init__(*args, **kwargs) #Calls all arguments in init from the superior class 'system'.
        
    def setup(self):
        """Aerodynamic inputs"""
        #Directory to the Aero results in CSV in order to build the interpolation functions.
        Aero_CSV = r'../../disciplines/aerodynamics/tools/Results/aero_results.csv'
        # Ranges to create CL and CD interpolation functions from AVL Aero Results (CSV file).
        # REMARK: To maintain units, array and split format to the values must correlate to the CSV file results.
        self.add_inward('alpha_list', list(np.arange(-6.0, 10.0,2)), unit='deg', desc='Range of AOA to create functions, the points shall match with the Aero CSV ranges')
        self.add_inward('mach_list', list(np.arange(0.0, 0.88,0.04)), unit='', desc='Range of Mach to create functions')
        self.add_inward('altitude_list', list(np.arange(0.0,12000,500)), unit='m', desc='Range of altitude to create functions')
        
        # Creation of functions for the aerodynamic coefficients interpolation.
        CLAeroIt = aeroInterp.CL_Interpolation_function(self.alpha_list,self.mach_list,self.altitude_list,Aero_CSV)
        CDAeroIt = aeroInterp.CD_Interpolation_function(self.alpha_list,self.mach_list,self.altitude_list,Aero_CSV)
        DAeroIt = aeroInterp.Drag_Interpolation_function(self.alpha_list,self.mach_list,self.altitude_list,Aero_CSV)

        ###-------------------------------------------------------
        ### Target distance and descent point computation
        ###-------------------------------------------------------
        # Algorithm to find descent point.
        # It stops until the error between the final AC position and the 
        # desired distance is less than the specified tolerance "distance error".
        
        self.add_inward('Target_mission_distance', 500.0, unit ="NM", desc='Target or expected distance for the mission in Nautic Miles')
        assert self.Target_mission_distance > 0, f"number greater than 0 expected, got: {self.Target_mission_distance}"
        self.add_inward('Cruise_distance', 10.0, unit ="NM", desc ='Current distance to travel in cruise segment, variable used to calculate the descent point')
        assert self.Cruise_distance > 0, f"number greater than 0 expected, got: {self.Cruise_distance}"
        assert self.Cruise_distance < self.Target_mission_distance, f"Cruise distance shall be lower than the target mission distance, got: {self.Cruise_distance} and {self.Target_mission_distance}"
        self.add_inward('dist_error', 50.0, unit ="ft", desc ='Calculated error to the Target_mission_distance, initialized at 50ft ')
        self.add_inward('dist_error_tol', 32.0, unit ="ft", desc ='Tolerance for the distance error, 32ft (aprox. 10m)')
        assert self.dist_error_tol < self.dist_error, f"The tolerance shall be lower than the distance error to initialize simulation, got: {self.dist_error_tol} and {self.dist_error}"

        ###-------------------------------------------------------
        ### Parse into SI for computation.
        ###-------------------------------------------------------
        self.Target_mission_distance = uc.nm2m(self.Target_mission_distance) #
        self.Cruise_distance = uc.nm2m(self.Cruise_distance) #
        self.dist_error = uc.ft2m(self.dist_error) #
        self.dist_error_tol = uc.ft2m(self.dist_error_tol) #
        
        ###-------------------------------------------------------
        ### Mission loop 
        ###-------------------------------------------------------
        # runs to find the correct descent point to reach the Target_mission_distance.
        while self.dist_error > self.dist_error_tol :
            # Geometry and Aero functions allocation.
            self.Mission = missionProfile.mission_profile(name='Mission', asb_aircraft_geometry=airplane_geom(), mission_callback=live_graph_callback)
            # Aero functions allocation.
            self.Mission.CLAeroIt = CLAeroIt
            self.Mission.CDAeroIt = CDAeroIt
            self.Mission.DAeroIt = DAeroIt
            # Cruise distance allocation
            self.Mission.Cruise_segment.Cruise_distance_target = self.Cruise_distance
            
            self.Mission.run_drivers() #Running mission drivers.
            #Fuel mass needed for the entire mission.
            for segment in self.Mission.flightSegments:
                self.Mission.Simulation_W_f = self.Mission.Simulation_W_f + segment.out_p.fuel_mass
            #Update of variables and error for descent distance.
            self.dist_error = abs(self.Target_mission_distance - self.Mission.Descent_segment_2.out_p.position[0])
            self.Cruise_distance = self.Mission.Cruise_segment.Cruise_distance_target + self.Target_mission_distance - self.Mission.Descent_segment_2.out_p.position[0]
        ###-------------------------------------------------------
        ### Feedback from the results. Logging function.
        ###-------------------------------------------------------
        if self.Mission.Acc_Mach.time > 300:
            logging.info('The acceleration takes a time of', self.Mission.Acc_Mach.time)
        elif self.Mission.Simulation_W_f > self.Mission.W_f:
            logging.info('The fuel max capacity is not enough for the required distance and the fuel required, Fuel capacity:', self.Mission.Simulation_W_f[0], 'Actual Max Fuel capacity :', self.Mission.W_f)
        elif abs(self.Mission.Climb_segment_2.in_p.position[2] - self.Mission.cruise_altitude) > 0.01 :
            logging.info('Flight level not reachable. Please check the data to verify the event at the altitude of', self.Mission.Climb_segment_2.out_p.position[2]) 

        self.Mission.drivers.clear()
                
    def compute(self):
        
        ###-------------------------------------------------------
        ### Mission data and time management for final resutls
        ###-------------------------------------------------------
        
        df_mission = pd.DataFrame() #Dataframe that will concatenate the info for the complete mission.
        
        #Saving each segment results and saving it in the data dictionary.
        for segment in self.Mission.flightSegments : 
            df_seg = self.Mission.drx[segment.name].recorder.export_data()            
            df_seg = df_seg.drop(df_seg.index[-2]) # The row [-2] shall be dropped since the results are repeated on CosApp format.
            df_seg['Segment'] = segment.name # Creating a segment column so a row corresponds to its segment.
            df_seg.rename(columns={'time [-]': 'Time [s]'},inplace=True) # Change of time into the correct units [s]
            df_seg['Time [min]'] = df_seg['Time [s]'].div(60) # Addition of time in minutes.
            df_mission=pd.concat([df_mission, df_seg])  # Concatenation of all the segments df into the mission df
            self.data[segment.name] = df_seg # Inserting the segments df into the the data dictionary.
        
        self.data['mission'] = df_mission # Inserting the mission df into the the data dictionary.
        
        
        """Algorithm to rearange the data contained in the mission data frame."""
        
        self.data['mission'].reset_index(drop=True,inplace=True) 
        data_copy = self.data['mission'].copy()
        t_sec=0
        t_min=0

        for index in range(1,len(data_copy['Time [s]']),1): 
            t_sec += max(0, (data_copy['Time [s]'][index] - data_copy['Time [s]'][index-1]))
            t_min+= max(0, (data_copy['Time [min]'][index] - data_copy['Time [min]'][index-1]))
            self.data['mission']['Time [s]'][index]=t_sec
            self.data['mission']['Time [min]'][index]=t_min
        
        ###-------------------------------------------------------
        ### Plots figure definition
        ###-------------------------------------------------------
        """This sets the methods in order to define the figure and its interactive tools.
           The plot will show those parameters chosen in the widgets"""
        
        _segment_opt =  list(self.data.keys())
        self._segment_widget =  widgets.Dropdown(value=_segment_opt[0], options=_segment_opt, width = 100)
        self._segment_widget.observe(self.display, "value")
        
        unwanted_keys= ['Section', 'Status', 'Error code', 'Reference','Segment'] # Keys to avoid for widgets options [Non-numerical]
        keys = list(set(self.data[self._segment_widget.value].keys()) - set(unwanted_keys))   

#         By default first element of the dataframe
        self._x_widget = widgets.Dropdown(value=keys[0], options=keys)
        self._x_widget.observe(self.display, "value")
        
#         By default second element of the dataframe
        self._y_widget = widgets.Dropdown(value=keys[1], options=keys)
        self._y_widget.observe(self.display, 'value')
        
#         By default third element of the dataframe
        self.add_parameter_widget = widgets.Dropdown(value=keys[2], options=keys)
        self.add_parameter_widget.observe(self.display, 'value')
        
    def _build_plots(self):
        
        x_name = self._x_widget.value
        y_name = self._y_widget.value
        add_name = self.add_parameter_widget.value
        
        seg_df = self.data[self._segment_widget.value]

        x_ = round(seg_df[x_name].astype(float),4)
        y_ = round(seg_df[y_name].astype(float),4)
        add_ = round(seg_df[add_name].astype(float),4)

        self._fig = px.line(seg_df,x=x_, y=y_,text=add_ ,color='Segment')
        
        #Algorithm to find and save the points to use in the plot.
        segs = seg_df['Segment'].unique() #Finding those segments used in the simulation (saved in the results).
        x_e=[] #List for x values matching the event
        y_e=[] #List for y values matching the event
        evv=[] #List for event labels
        for seg in segs:
            for elm in seg_df['Reference']: #Elements in 'Reference' columns
                if search(seg,elm):
                    # Finding points where the segment name corresponds to the element in 'Reference' containing the event, i.e.,
                    # for all the segments in the mission column search those matching the event name in the 'Reference' column.
                    # For instance, the segment 'Cruise_segment' will be found in the name of the event 'Cruise_segment.cruise_distance_reached' 
                    x_e.append(round(seg_df.loc[seg_df['Reference'] == elm, x_name].values.astype(float)[0],4)) 
                    y_e.append(round(seg_df.loc[seg_df['Reference'] == elm, y_name].values.astype(float)[0],4))
                    # Format of the event label to show in the plot for visualization pruposes.
                    evv.append(elm.replace(seg+'.', ''))
                    
        self._fig.update_traces(mode="markers+lines", hovertemplate = x_name+': %{x:.4f} <br>'+y_name+'%{y:.4f}<br>'
                   +add_name+'%{text:.4f}<br>')

        self._fig.add_traces(
            go.Scatter(
                text=evv, x=x_e, y=y_e, mode="markers", name="Events", marker=dict(size=8)
        ))
        
        self._fig = go.FigureWidget(self._fig)
        self._fig.update_layout(
        title_text=self._segment_widget.value, title_x=0.5,
            xaxis_title=x_name,
            yaxis_title=y_name,
         )

    def display(self, change=None) -> display:
        
        """ Display the user interface: return the display object"""
        
        clear_output(wait=True)
        self._update_plots()

        toolbar = widgets.HBox(
        [widgets.Label(value="x:"), self._x_widget,
         widgets.Label(value="y:"), self._y_widget,
         widgets.Label(value="segment:"), self._segment_widget,
         widgets.Label(value="Add parameter"), self.add_parameter_widget
        ])

        ui = widgets.VBox([toolbar, self._fig])
        return display(ui)
    
    def _update_plots(self):
        """
        Update the plots
        """
        self._fig = None
        self._build_plots()
        
    def export_csv(self, segment):
        
        """ The user can chose which segment data to export into CSV.
            It is important to define the workspace directory on which the data will be saved"""
        
        if segment in self.data.keys():
            self.data[segment].to_csv('amad/disciplines/performance/systems/CSV_Results/'+segment+'_data.csv', encoding='utf-8', index=False)
            print('The segment ' + segment + 'for a range of ', uc.m2nm(self.Target_mission_distance),' NM has been exported succesfully')
        else:
            print('The segment name is not found in the Mission profile')

In [ ]:
live_figure

In [ ]:
start_time = time.perf_counter()
Mission_sim = Mission_simulation('Mission_sim') #By creating the object mission_sim it starts to run the simulation.
end_time = time.perf_counter()
print('time=',end_time - start_time)

In [ ]:
Mission_sim.run_once() #The system compute method lauched by "run_once" manage the the post-processing data.

In [ ]:
Mission_sim.display() # Method to display the interactive figure

In [ ]:
Mission_sim.export_csv('mission') # Method to export the data of a desired segment.

In [ ]:
Mission_sim.Mission.S

In [ ]:
Mission_sim.Mission['Climb_segment_1'].children['enginePerfo'].inwards

In [ ]:
Mission_sim.Mission